In [ ]:
!gdown --id 1-40b21KUaWWpfgFS2ZWDSm9fOfGLdjkg


Downloading...
From: https://drive.google.com/uc?id=1-40b21KUaWWpfgFS2ZWDSm9fOfGLdjkg
To: /content/tools.zip
925MB [00:07, 138MB/s]


In [ ]:
!unzip tools.zip

Archive:  tools.zip
  inflating: model_trained_after_1_epochs.bin  
  inflating: model_trained_after_2_epochs.bin  
   creating: static/
   creating: static/css/
   creating: static/.ipynb_checkpoints/
   creating: templates/
  inflating: templates/actor.html    
   creating: templates/.ipynb_checkpoints/
  inflating: templates/404.html      
  inflating: templates/500.html      
  inflating: templates/index.html    


In [ ]:
# !gdown --id 1Hk_8a5-ymyZTb6yyqJTdIHuS-j7rYVIz
# !gdown --id 1HiJNMsgeSwu9GEzgkxiKh-Z7Vi2NplGj


510MB [00:03, 149MB/s]


In [ ]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 9.3 MB/s 
     |████████████████████████████████| 636 kB 67.3 MB/s 
     |████████████████████████████████| 3.3 MB 43.2 MB/s 
     |████████████████████████████████| 895 kB 52.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import re
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
import random
from transformers import GPT2Tokenizer
from tqdm import tnrange
from transformers import GPT2LMHeadModel
def deEmojify(string):
  emoji_pattern = re.compile("["
                              u"\U0001F600-\U0001F64F"  # emoticons
                              u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                              u"\U0001F680-\U0001F6FF"  # transport & map symbols
                              u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                              u"\U00002500-\U00002BEF"  # chinese char
                              u"\U00002702-\U000027B0"
                              u"\U00002702-\U000027B0"
                              u"\U000024C2-\U0001F251"
                              u"\U0001f926-\U0001f937"
                              u"\U00010000-\U0010ffff"
                              u"\u2640-\u2642"
                              u"\u2600-\u2B55"
                              u"\u200d"
                              u"\u23cf"
                              u"\u23e9"
                              u"\u231a"
                              u"\ufe0f"  # dingbats
                              u"\u3030"
                              "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)


# Processing and cleaning the sentences
def clean_text(article):
  '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''

  # Format words and remove unwanted characters
  article = re.sub(r'http\S+', '', article)
  article = re.sub(r'WWW\S+', '', article)
  article = re.sub(r'www\S+', '', article)
  # article = re.sub('[#|®™©*/@,:=+?"()_{}“”;<>\[\]\']','',article)
  article = deEmojify(article)
  article = re.sub(r'\<a href', ' ', article)
  article = re.sub(r'&amp;', '', article)
  article = re.sub(r'<br />', ' ', article)
  article = re.sub(r'<br >', ' ', article)
  article = ' '.join(article.split())
  return article
def add_special_tokens():
	""" Returns GPT2 tokenizer after adding separator and padding tokens """
	tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
	special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>','eos_token':'<|end|>'}
	num_add_toks = tokenizer.add_special_tokens(special_tokens)
	return tokenizer
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
  assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
  top_k = min(top_k, logits.size(-1))  # Safety check
  if top_k > 0:
    # Remove all tokens with a probability less than the last token of the top-k
    indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
    logits[indices_to_remove] = filter_value

  if top_p > 0.0:
    sorted_logits, sorted_indices = torch.sort(logits, descending=True)
    cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

    # Remove tokens with cumulative probability above the threshold
    sorted_indices_to_remove = cumulative_probs > top_p
    # Shift the indices to the right to keep also the first token above the threshold
    sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
    sorted_indices_to_remove[..., 0] = 0

    indices_to_remove = sorted_indices[sorted_indices_to_remove]
    logits[indices_to_remove] = filter_value
  return logits

def make_sample(article):
  tokenizer = add_special_tokens()
  article = clean_text(article)
  article = tokenizer.encode(article)
  article = article[:950]
  text = tokenizer.encode(tokenizer.pad_token)*1024
  post = tokenizer.encode(tokenizer.pad_token)*951
  post[:len(article)] = article
  content = post + tokenizer.encode(tokenizer.eos_token)
  text[:len(content)] = content
  text = torch.tensor(text)
  sample = {'article': text, 'sum_idx': len(post)}
  return sample

  
def sample_seq(model, context, length, temperature=1, top_k=0, top_p=0.0,device=torch.device('cpu')):
  """ Generates a sequence of tokens 
      Args:
          model: gpt/gpt2 model
          context: tokenized text using gpt/gpt2 tokenizer
          length: length of generated sequence.
          device: torch.device object.
          temperature >0: used to control the randomness of predictions by scaling the logits before applying softmax.
          top_k > 0: keep only top k tokens with highest probability (top-k filtering).
          top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
  """
  
  context = torch.tensor(context, dtype=torch.long,device=device)
  context = context.unsqueeze(0)
  generated = context
  with torch.no_grad():  
    for _ in tnrange(length):
      inputs = {'input_ids': generated}
      outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
      next_token_logits = outputs[0][0, -1, :] / temperature
      filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
      next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
      generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
  return generated


def generate_sample(sample,  model, tokenizer=add_special_tokens(),num=1, length=60,  temperature=1, top_k=10, top_p=0.5,device=torch.device('cpu')):
  """ Generate summaries for "num" number of articles.
      Args:
          data = GPT21024Dataset object
          tokenizer = gpt/gpt2 tokenizer
          model = gpt/gpt2 model
          num = number of articles for which summaries has to be generated
          eval_step = can be True/False, checks generating during evaluation or not
  """

  idx = sample['sum_idx']
  context = sample['article'][:idx].tolist()
  # summary = sample['article'][idx+1:].tolist()
  generated_text = sample_seq(model, context, length,  temperature, top_k, top_p,device=device)
  generated_text = generated_text[0, len(context):].tolist()
  text = tokenizer.convert_ids_to_tokens(generated_text, skip_special_tokens=False)
  text = tokenizer.convert_tokens_to_string(text)
  matches_positions = [i for i in range(len(text)) if text.startswith('<|end|>', i)]
  if len(matches_positions) > 0:
    pos = matches_positions[0];
    if len(text[:pos].split()) < 20:
      if len(matches_positions) > 1:
        pos = matches_positions[1];
        if len(text[:pos].split()) < 20:
          if len(matches_positions) > 2:
            pos = matches_positions[2];
    text = text[:pos]

  article = tokenizer.decode(context).replace('<|pad|>', ' ').replace('<|sep|>', ' ').replace('<|end|>', ' ')
  # summary = tokenizer.decode(summary).replace('<|pad|>', ' ').replace('<|sep|>', ' ').replace('<|end|>', ' ')
  text = text.replace('<|pad|>', ' ').replace('<|sep|>', ' ').replace('<|end|>', ' ')
  # df.append([article, text, summary])
  return text

def model(path,device=torch.device('cuda')):
  tokenizer = add_special_tokens()
  model = GPT2LMHeadModel.from_pretrained('gpt2')
  model.to(device)
  model.resize_token_embeddings(len(tokenizer))
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint)
  return model

def get_result(text,model,device=torch.device('cuda'),length=60):
  article=make_sample(text)
  return generate_sample(article,model,device=device,length=length, temperature=0.7, top_k=25, top_p=0.95)

In [ ]:
model_1 =model('model_trained_after_2_epochs.bin')

In [ ]:
!pip install flask-ngrok
!pip install Flask
!pip install Flask-WTF
!pip install Flask-Bootstrap4
!pip install flask_debugtoolbar

     |████████████████████████████████| 169 kB 8.4 MB/s 
     |████████████████████████████████| 1.7 MB 7.7 MB/s 
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3943 sha256=c6d88d93f151af647d5ec49e9702b8069942f149b758cd3a2d768407eccd7bc9
  Stored in directory: /root/.cache/pip/wheels/64/34/11/053f47218984c9a31a00f911ed98dda036b867481dcc527a12
Successfully built visitor
     |████████████████████████████████| 325 kB 9.8 MB/s 
     |████████████████████████████████| 111 kB 69.3 MB/s 
  Created wheel for Blinker: filename=blinker-1.4-py3-none-any.whl size=13479 sha256=42d906df7712b2794c4a9eabc872ca7cd5920a903009b8b4b3086c7c9de8cbd7
  Stored in directory: /root/.cache/pip/wheels/22/f5/18/df711b66eb25b21325c132757d4314db9ac5e8dabeaf196eab
Successfully built Blinker


In [ ]:
from flask import Flask, render_template, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import Flask, render_template
from flask_debugtoolbar import DebugToolbarExtension

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

In [ ]:


# Flask-WTF requires an encryption key - the string can be anything
app.config['SECRET_KEY'] = 'C2HWGVoMGfNTBsrYQg8EcMrdTimkZfAb'

# Flask-Bootstrap requires this line
Bootstrap(app)

In [ ]:
class NameForm(FlaskForm):
  name = StringField('Article', validators=[DataRequired()])
  submit = SubmitField('Submit')
# functions to be used by the routes

# # retrieve all the names from the dataset and put them into a list
# def get_result(text,device=torch.device('cuda'),length=60):
#   return genrate_sample(text,device=device,length=length)

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def index():
  form = NameForm()
  if form.validate_on_submit():
    article = form.name.data
    oneliner=get_result(article,model_1)
    form.name.data=''
    return render_template( 'actor.html', oneliner=oneliner)
  return render_template('index.html', form=form)

@app.route('/actor/',methods=['GET', 'POST'])
def actor(oneliner):
  # run function to get actor data based on the id in the path
  if oneliner == "":
    # redirect the browser to the error template
    return render_template('404.html'), 404
  else:
    # pass all the data for the selected actor to the template
    return render_template('actor.html', oneliner=oneliner)

# 2 routes to handle errors - they have templates too

@app.errorhandler(404)
def page_not_found(e):
  return render_template('404.html'), 404

@app.errorhandler(500)
def internal_server_error(e):
  return render_template('500.html'), 500

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2438284dcba8.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [08/Aug/2021 09:49:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2021 09:49:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Aug/2021 09:49:59] "GET /favicon.ico HTTP/1.1" 404 -
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:110: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`


127.0.0.1 - - [08/Aug/2021 09:50:27] "POST / HTTP/1.1" 200 -


127.0.0.1 - - [08/Aug/2021 09:53:03] "GET / HTTP/1.1" 200 -


127.0.0.1 - - [08/Aug/2021 09:53:29] "POST / HTTP/1.1" 200 -


127.0.0.1 - - [08/Aug/2021 09:53:44] "GET / HTTP/1.1" 200 -


127.0.0.1 - - [08/Aug/2021 09:54:06] "POST / HTTP/1.1" 200 -
